<img src="img/Marca-ITBA-Color-ALTA.png" width="250">

# Programación para el Análisis de Datos

## Clase 4 parte 3 - Joins

#### Referencias y bibliografía de consulta:

- Python for Data Analysis by Wes McKinney (O’Reilly) 2018 - capítulo 8

- https://pandas.pydata.org/

### Introducción

Frecuentemente, los datos estarán dispersos en varios archivos o tablas de bases de datos o están organizados de forma que no sea fácil de analizar. Por este motivo, `Pandas` incorpora métodos que faciliten esta tarea.

En esta clase vamos a ver las herramientas para ayudar a combinar, unir y reorganizar los datos.

Antes, es necesario repasar el concepto de indexación múltiple e indexación jerárquica.

### 1- Indexing jerárquico

La indexación jerárquica es una característica importante de `Pandas` que permite tener múltiples (dos o más) niveles de índice en un eje. De forma algo abstracta, proporciona una forma de trabajar con datos de dimensiones superiores en una forma de dimensiones inferiores. 

Comencemos con un ejemplo simple: crear una serie con una lista de listas como `index`

In [1]:
import numpy as np
import pandas as pd

In [3]:
series1 = pd.Series(np.random.randn(9),
                    index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                           [1, 2, 3, 1, 3, 1, 2, 2, 3]])

series1

a  1   -2.065035
   2    0.703762
   3    0.831307
b  1   -2.255012
   3   -0.020260
c  1    0.938980
   2    2.016455
d  2    0.469549
   3   -0.975703
dtype: float64

In [4]:
series1.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [5]:
series1.loc[('a', 2)]

0.7037621272185538

Con un objeto indexado jerárquicamente, es posible realizar una indexación parcial, que permite seleccionar de forma concisa subconjuntos de datos:

In [6]:
series1['b']

1   -2.255012
3   -0.020260
dtype: float64

In [7]:
series1['b':'c']

b  1   -2.255012
   3   -0.020260
c  1    0.938980
   2    2.016455
dtype: float64

La selección es incluso posible desde un nivel "interno":

In [8]:
series1.loc[:, 2]

a    0.703762
c    2.016455
d    0.469549
dtype: float64

In [9]:
series1.loc[['a', 'b'], [1,2]]

a  1   -2.065035
   2    0.703762
b  1   -2.255012
dtype: float64

In [10]:
series1.loc[['a', 'b'], [1,3]]

a  1   -2.065035
   3    0.831307
b  1   -2.255012
   3   -0.020260
dtype: float64

In [11]:
series1.loc[['a', 'b'], 1:3]

a  1   -2.065035
   2    0.703762
   3    0.831307
b  1   -2.255012
   3   -0.020260
dtype: float64

El indexing jerárquico desempeña un papel importante en el reshaping de los datos y las operaciones basadas en grupos, como la formación de una tabla pivot. Por ejemplo, se podrían reorganizar los datos en un `DataFrame` utilizando su método `unstack`: 

In [12]:
series1

a  1   -2.065035
   2    0.703762
   3    0.831307
b  1   -2.255012
   3   -0.020260
c  1    0.938980
   2    2.016455
d  2    0.469549
   3   -0.975703
dtype: float64

In [13]:
series1.unstack()

,1,2,3
a,-2.065035,0.703762,0.831307
b,-2.255012,NaN,-0.020260
c,0.938980,2.016455,NaN
d,NaN,0.469549,-0.975703


In [14]:
series1.unstack().stack()

a  1   -2.065035
   2    0.703762
   3    0.831307
b  1   -2.255012
   3   -0.020260
c  1    0.938980
   2    2.016455
d  2    0.469549
   3   -0.975703
dtype: float64

Con un `DataFrame`, cualquiera de los ejes puede tener un índice jerárquico:

In [15]:
df1 = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])

df1

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

Los niveles jerárquicos pueden tener nombres.

In [16]:
df1.index.names = ['key1', 'key2']
df1.columns.names = ['state', 'color']
df1

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

Con la indexación parcial de columnas se pueden seleccionar grupos de columnas de manera similar:

In [16]:
df1['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [56]:
df1.loc[:, [('Ohio','Red'),('Colorado','Green')]]

state     Ohio Colorado
color      Red    Green
key1 key2              
a    1       1        2
     2       4        5
b    1       7        8
     2      10       11

In [18]:
# Usando el método .xs()
df1.xs('Green', level=1, axis=1)


state      Ohio  Colorado
key1 key2                
a    1        0         2
     2        3         5
b    1        6         8
     2        9        11

In [50]:
df1.xs('a', level=0, axis=0)

state  Ohio     Colorado
color Green Red    Green
key2                    
1         0   1        2
2         3   4        5

####  Re-organización y Re-sorting de indices

El método `swaplevel()` toma dos números o nombres de nivel y devuelve un nuevo objeto con los niveles intercambiados (pero por lo demás los datos no se modifican):

In [57]:
df1.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [61]:
df1.swaplevel('color', 'state', axis=1)

color     Green  Red    Green
state      Ohio Ohio Colorado
key1 key2                    
a    1        0    1        2
     2        3    4        5
b    1        6    7        8
     2        9   10       11

El método `sort_index()`, por otro lado, ordena los datos usando sólo los valores de un determinado nivel. Cuando se intercambian niveles, no es raro que también se utilice `sort_index()` para que el resultado se ordene lexicográficamente por el nivel indicado:

In [62]:
df1.sort_index(level=0, ascending=True)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [63]:
df1.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 2- Combinación y unión de datasets

Para agregar información a un dataset (`DataFrame`), se pueden incluir filas o columnas. 
Dentro del módulo `Pandas` existen 3 métodos para combinar `DataFrames`:
 - `pandas.merge()` conecta filas en `DataFrames` utilizando columnas o índices para alinearlos. Este método es muy similar al `join`, pero es mas versatil, ya que permite relacionar `DataFrames` utilizando columnas diferentes. 
 - `pandas.concat()` concatena o "apila" objetos a lo largo de un eje. Este método permite actualizar los indices o manterlos al concatenar.
  - `pandas.DataFrame.join()` conecta filas de `DataFrames` alineando el índice o alguna columna de uno con el índice del otro. Esto será familiar para los usuarios de SQL u otras bases de datos relacionales, ya que implementa operaciones de `join` de bases de datos.
 
 Veamos un pequeño ejemplo de como se utilizan estas funciones.

In [23]:
df2 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [24]:
df3 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

df3

,key,data2
0,a,0
1,b,1
2,d,2


Este es un ejemplo de una unión de muchos a uno; los datos de **df2** tienen múltiples filas etiquetadas como **a** y **b**, mientras que **df3** tiene sólo una fila para cada valor en la columna clave. Llamando a `merge()` con estos objetos obtenemos: 

In [25]:
pd.merge(df2, df3)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


Hay que notar que no especificamos sobre cuál columna realizar el `merge()`. Si no se especifica esa información, el método `pd.merge()` utiliza los nombres de las columnas superpuestas como claves. Sin embargo, es una buena práctica especificar explícitamente:

In [26]:
pd.merge(df2, df3, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


Si los nombres de las columnas son diferentes en cada objeto, podemos especificarlos por separado:

In [27]:
df4 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

df5 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})

display(df4)
display(df5)

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,rkey,data2
0,a,0
1,b,1
2,d,2


In [28]:
pd.merge(df4, df5, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


Podemos notar que los valores 'c' y 'd' y los datos asociados faltan en el resultado. Por default, `pd.merge()` realiza un `'inner'` join. Las claves del resultado son la intersección, o el conjunto común que se encuentra en ambas tablas. Otras opciones posibles son `'left'`, `'right'` y `'outer'`. El  `'outer'` join toma la unión de las claves, combinando el efecto de aplicar ambas uniones, `'left'` y `'right'`:

In [29]:
pd.merge(df2, df3, how='outer')

,key,data1,data2
0,a,2.0,0.0
1,a,4.0,0.0
2,a,5.0,0.0
3,b,0.0,1.0
4,b,1.0,1.0
5,b,6.0,1.0
6,c,3.0,NaN
7,d,NaN,2.0


###  3) Ejemplo de caso de uso
Veamos ahora un ejemplo de caso de uso de una base de datos de telefonía celular. 

In [30]:
user_usage = pd.read_csv("data/user_usage.csv")
user_device = pd.read_csv("data/user_device.csv")
devices = pd.read_csv("data/android_devices.csv")
devices.rename(columns={"Retail Branding": "manufacturador"}, inplace=True)

In [31]:
user_usage.head()

,min_saliente_mes,sms_saliente_mes,data_mb_mes,id_uso
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


In [32]:
user_device.head()

,id_uso,id_usuario,plataforma,plataforma_version,dispositivo,id_tipo_uso
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


In [33]:
devices.head(10)

,manufacturador,Marketing Name,Dispositivo,Modelo,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,AD681H,Smartfren Andromax AD681H,NaN,NaN,NaN,NaN
1,NaN,NaN,FJL21,FJL21,NaN,NaN,NaN,NaN
2,NaN,NaN,T31,Panasonic T31,NaN,NaN,NaN,NaN
3,NaN,NaN,hws7721g,MediaPad 7 Youth 2,NaN,NaN,NaN,NaN
4,3Q,OC1020A,OC1020A,OC1020A,NaN,NaN,NaN,NaN
5,7Eleven,IN265,IN265,IN265,NaN,NaN,NaN,NaN
6,A.O.I. ELECTRONICS FACTORY,A.O.I.,TR10CS1_11,TR10CS1,NaN,NaN,NaN,NaN
7,AG Mobile,AG BOOST 2,BOOST2,E4010,NaN,NaN,NaN,NaN
8,AG Mobile,AG Flair,AG_Flair,Flair,NaN,NaN,NaN,NaN
9,AG Mobile,AG Go Tab Access 2,AG_Go_Tab_Access_2,AG_Go_Tab_Access_2,NaN,NaN,NaN,NaN


#### Primer Merge

Intentemos analizar cuáles consumos existen para cada tipo de dispositivo diferente.

Para esto necesitamos relacionar mediante el código de uso `id_uso` las tablas `user_usage` y `user_device`.



In [34]:
result = pd.merge(user_usage,
                  user_device[['id_uso', 'plataforma', 'dispositivo']],
                  on='id_uso')
result.head()

,min_saliente_mes,sms_saliente_mes,data_mb_mes,id_uso,plataforma,dispositivo
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


En este merge se pueden ver los datos de ambas tablas, unidos por la columna `id_uso`.
Analicemos en profundidad que es lo que sucedió durante el merge.

In [35]:
print("user_usage dimensions: {}".format(user_usage.shape))
print("user_device dimensions: {}".format(user_device[['id_uso', 'plataforma', 'dispositivo']].shape))
print("result dimensions: {}".format(result.shape))

user_usage dimensions: (240, 4)
user_device dimensions: (272, 3)
result dimensions: (159, 6)


Vemos que la cantidad de datos no es la misma. Esto se debe a que se realizó un `inner join`. Esto significa que las **claves** que no se encuentran en ambas tablas, se descartan.

In [36]:
user_usage['id_uso'].isin(user_device['id_uso']).value_counts()

id_uso
True     159
False     81
Name: count, dtype: int64

#### Ejemplo Left merge

El `left merge` o `left join` permite que se conserven todas las filas de uno de los 2 `DataFrames`.
En este caso, la tabla de la izquierda es la que se llama al método `merge` y la tabla de la derecha (`right`) es la que se utiliza como argumento de la función.
En este caso, `left=user_usage` y `right=user_device`.
Aplicando un left join, las columnas de la tabla derecha contendra `NaN` en todas las filas correspondientes a claves que no tienen su par en ambas tablas.

In [37]:
result = pd.merge(user_usage,
                  user_device[['id_uso', 'plataforma', 'dispositivo']],
                  on='id_uso', how='left')

print("user_usage dimensions: {}".format(user_usage.shape))
print("result dimensions: {}".format(result.shape))

print("Hay {} valores faltantes en el resultado.".format(
        result['dispositivo'].isnull().sum()))

user_usage dimensions: (240, 4)
result dimensions: (240, 6)
Hay 81 valores faltantes en el resultado.


In [38]:
result.head()

,min_saliente_mes,sms_saliente_mes,data_mb_mes,id_uso,plataforma,dispositivo
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


In [39]:
result.tail()

,min_saliente_mes,sms_saliente_mes,data_mb_mes,id_uso,plataforma,dispositivo
235,260.66,68.44,896.96,25008,NaN,NaN
236,97.12,36.50,2815.00,25040,NaN,NaN
237,355.93,12.37,6828.09,25046,NaN,NaN
238,632.06,120.46,1453.16,25058,NaN,NaN
239,488.70,906.92,3089.85,25220,NaN,NaN


#### Ejemplo Right merge

El `right merge` o `right join` entre 2 `DataFrames` mantiene todas las filas correspondientes al `DataFrame` de la derecha, mientras que las columnas de la tabla izquierda de los registros que se encuentren en la tabla derecha, pero no en la izquierda, se completaran con `NaN`.

In [40]:
result = pd.merge(user_usage,
                 user_device[['id_uso', 'plataforma', 'dispositivo']],
                 on='id_uso', how='right')

print("user_device dimensions: {}".format(user_device.shape))
print("result dimensions: {}".format(result.shape))
print("Hay {} valores faltantes en la columna 'data_mb_mes' del resultado.".format(
        result['data_mb_mes'].isnull().sum()))
print("Hay {} valores faltantes en la columna 'plataforma' del resultado.".format(
        result['plataforma'].isnull().sum()))

user_device dimensions: (272, 6)
result dimensions: (272, 6)
Hay 113 valores faltantes en la columna 'data_mb_mes' del resultado.
Hay 0 valores faltantes en la columna 'plataforma' del resultado.


#### Ejemplo Outer merge

Un `full outer join` o `outer merge` mantiene todos los registros de ambos `DataFrames` en el resultado.
Las filas se alinearan en donde se compartan claves, y el resto de los registros tendran nulos en las columnas del `DataFrame` que corresponda.

En el resultado final, un subset de filas no tendran valores faltantes. Estos registros, que encontraron un match en la clave del otro `DataFrame` corresponden al resultado obrenido en el `inner merge`.

In [41]:
print("Hay {} valores unicos para ese id_uso en los DataFrames.".format(
        pd.concat([user_usage['id_uso'], user_device['id_uso']]).unique().shape[0]))

result = pd.merge(user_usage,
                 user_device[['id_uso', 'plataforma', 'dispositivo']],
                 on='id_uso', how='outer', indicator=True)

print("El Outer merge tiene {} registros.".format(result.shape))

print("Hay {} filas sin valores faltantes.".format(
    (result.apply(lambda x: x.isnull().sum(), axis=1) == 0).sum()))

Hay 353 valores unicos para ese id_uso en los DataFrames.
El Outer merge tiene (353, 7) registros.
Hay 159 filas sin valores faltantes.


In [42]:
result

,min_saliente_mes,sms_saliente_mes,data_mb_mes,id_uso,plataforma,dispositivo,_merge
0,NaN,NaN,NaN,22782,ios,"iPhone7,2",right_only
1,NaN,NaN,NaN,22783,android,Nexus 5,right_only
2,NaN,NaN,NaN,22784,android,SM-G903F,right_only
3,NaN,NaN,NaN,22785,ios,"iPhone7,2",right_only
4,NaN,NaN,NaN,22786,android,ONE E1003,right_only
...,...,...,...,...,...,...,...
348,260.66,68.44,896.96,25008,NaN,NaN,left_only
349,97.12,36.50,2815.00,25040,NaN,NaN,left_only
350,355.93,12.37,6828.09,25046,NaN,NaN,left_only
351,632.06,120.46,1453.16,25058,NaN,NaN,left_only


A continuación se muestra una imagen que representa los distintos `joins` mediante diagramas de Venn.
<img src="img/joins.jpg" width="500">

#### Merge Final - sumarizando productores de dispositivos


In [43]:
result = pd.merge(user_usage,
                 user_device[['id_uso', 'plataforma', 'dispositivo']],
                 on='id_uso',
                 how='left')

In [44]:
result = pd.merge(result, 
                  devices[['manufacturador', 'Modelo']],
                  left_on='dispositivo',
                  right_on='Modelo',
                  how='left')

result.head()                            

,min_saliente_mes,sms_saliente_mes,data_mb_mes,id_uso,plataforma,dispositivo,manufacturador,Modelo
0,21.97,4.82,1557.33,22787,android,GT-I9505,Samsung,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F,Samsung,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F,Samsung,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303,Sony,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F,Samsung,SM-G361F


#### Calculando estadísticas sobre el resultado final

Habiendo realizado los merges, se pueden calcular las estadisticas realizando un `groupby` sobre el manufacturador del dispositivo.

In [45]:
result.groupby("manufacturador").agg({
        "min_saliente_mes": "mean",
        "sms_saliente_mes": "mean",
        "data_mb_mes": "mean",
        "id_uso": "count"
    })

,min_saliente_mes,sms_saliente_mes,data_mb_mes,id_uso
manufacturador,,,,
HTC,299.842955,93.059318,5144.077955,44
Huawei,81.526667,9.500000,1561.226667,3
LGE,111.530000,12.760000,1557.330000,2
Lava,60.650000,261.900000,12458.670000,2
Lenovo,215.920000,12.930000,1557.330000,2
Motorola,95.127500,65.666250,3946.500000,16
OnePlus,354.855000,48.330000,6575.410000,6
Samsung,191.010093,92.390463,4017.318889,108
Sony,177.315625,40.176250,3212.000625,16


<!-- <span style="font-size:1.5em">Fin de la clase.</span> -->

<span style="font-size:2em">Muchas gracias por su atención!</span>